# Synchronous Document Translation
https://learn.microsoft.com/en-us/azure/ai-services/translator/document-translation/quickstarts/synchronous-rest-api


<img src="https://techcommunity.microsoft.com/t5/image/serverpage/image-id/552407i90F58669E536FA1A/image-size/large?v=v2&px=999">

> https://aka.ms/TranslatorLanguageCodes 

In [1]:
import datetime
import os
import requests
import sys
import time

from dotenv import load_dotenv
from IPython.display import FileLink

In [2]:
sys.version

'3.10.11 (main, May 16 2023, 00:28:57) [GCC 11.2.0]'

In [3]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 10-Apr-2024 09:28:19


## Azure AI Translator credentials

In [4]:
load_dotenv("azure.env")

key = os.getenv("AZURE_AI_TRANSLATION_KEY")
endpoint = os.getenv("AZURE_AI_TRANSLATION_ENDPOINTDOCUMENT")
region = os.getenv("AZURE_AI_TRANSLATION_REGION")

In [5]:
source_dir = "source"
target_dir = "translated"

os.makedirs(source_dir, exist_ok=True)
os.makedirs(target_dir, exist_ok=True)

In [6]:
def translate(input_file, sourceLanguage, targetLanguage):
    """
    Document translation
    """
    start = time.time()
    print(f"Translating the document from {sourceLanguage} to {targetLanguage} ...")

    params = {
        "sourceLanguage": sourceLanguage,
        "targetLanguage": targetLanguage,
        "api-version": "2023-11-01-preview",
    }
    
    path = "translator/document:translate"
    url = endpoint + path

    headers = {"Ocp-Apim-Subscription-Key": key}
    
    with open(input_file, "rb") as document:
        # Define the data to be sent
        # Find list of supported content types here: https://aka.ms/dtsync-content-type
        data = {
            "document": (os.path.basename(input_file),
                         document,
                         "application/vnd.openxmlformats-officedocument.wordprocessingml.document")
        }

        # Send the POST request
        response = requests.post(url, headers=headers, files=data, params=params)

    # Write the response content to a file
    with open(output_file, "wb") as output_document:
        output_document.write(response.content)

    elapsed = time.time() - start
    print("\nDone. Elapsed time: " + time.strftime("%H:%M:%S.{}".format(str(elapsed %
          1)[2:])[:15], time.gmtime(elapsed)))

## Test 1

In [7]:
sourceLanguage = "en"
targetLanguage = "fr"

In [8]:
input_file = os.path.join(source_dir, "worddocument.docx")
output_file = os.path.join(target_dir, "worddocument_translated_" + targetLanguage + ".docx")

In [9]:
source_link = FileLink(path=os.path.join(
    source_dir, os.path.basename(input_file)))
source_link

/mnt/batch/tasks/shared/LS_root/mounts/clusters/seretkow9/code/Users/seretkow/Azure AI Translator examples/source/worddocument.docx

In [10]:
translate(input_file, sourceLanguage, targetLanguage)

Translating the document from en to fr ...

Done. Elapsed time: 00:00:00.384228


In [11]:
translated_link = FileLink(path=os.path.join(
    target_dir, os.path.basename(output_file)))
translated_link

/mnt/batch/tasks/shared/LS_root/mounts/clusters/seretkow9/code/Users/seretkow/Azure AI Translator examples/translated/worddocument_translated_fr.docx

## Test 2

In [12]:
sourceLanguage = "en"
targetLanguage = "it"

In [13]:
input_file = os.path.join(source_dir, "worddocument.docx")
output_file = os.path.join(target_dir, "worddocument_translated_" + targetLanguage + ".docx")

In [14]:
translate(input_file, sourceLanguage, targetLanguage)

Translating the document from en to it ...

Done. Elapsed time: 00:00:00.426173


In [15]:
translated_link = FileLink(path=os.path.join(
    target_dir, os.path.basename(output_file)))
translated_link

/mnt/batch/tasks/shared/LS_root/mounts/clusters/seretkow9/code/Users/seretkow/Azure AI Translator examples/translated/worddocument_translated_it.docx

## Test 3

In [16]:
sourceLanguage = "en"
targetLanguage = "ar"

In [17]:
input_file = os.path.join(source_dir, "worddocument.docx")
output_file = os.path.join(target_dir, "worddocument_translated_" + targetLanguage + ".docx")

In [18]:
translate(input_file, sourceLanguage, targetLanguage)

Translating the document from en to ar ...

Done. Elapsed time: 00:00:00.440634


In [19]:
translated_link = FileLink(path=os.path.join(
    target_dir, os.path.basename(output_file)))
translated_link

/mnt/batch/tasks/shared/LS_root/mounts/clusters/seretkow9/code/Users/seretkow/Azure AI Translator examples/translated/worddocument_translated_ar.docx

## Test 4

In [20]:
sourceLanguage = "en"
targetLanguage = "zh-Hans"

In [21]:
input_file = os.path.join(source_dir, "worddocument.docx")
output_file = os.path.join(target_dir, "worddocument_translated_" + targetLanguage + ".docx")

In [22]:
translate(input_file, sourceLanguage, targetLanguage)

Translating the document from en to zh-Hans ...

Done. Elapsed time: 00:00:00.391599


In [23]:
translated_link = FileLink(path=os.path.join(
    target_dir, os.path.basename(output_file)))
translated_link

/mnt/batch/tasks/shared/LS_root/mounts/clusters/seretkow9/code/Users/seretkow/Azure AI Translator examples/translated/worddocument_translated_zh-Hans.docx